<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2020cnps_tSNE_for_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec を tSNE 視覚化

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 09/Mar/2020<br>
 MIT license
</div>

References:
- Paper: http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf
- Distill: https://distill.pub/2016/misread-tsne/
- wikipedia: https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding
- wikipedia.ja: https://ja.wikipedia.org/wiki/T%E5%88%86%E5%B8%83%E5%9E%8B%E7%A2%BA%E7%8E%87%E7%9A%84%E8%BF%91%E5%82%8D%E5%9F%8B%E3%82%81%E8%BE%BC%E3%81%BF%E6%B3%95



In [ ]:
# 2025 年 05 月 01 日時点でのメモ
# numpy のバージョンを強制的に 1.26.4 にダウングレードした場合，ランタイムの再起動をしないと
# ダウングレードした numpy が実行時に反映されない。
# このため一度このセルを実行した後に,上のメニューバー左から 5 つ目の「ランタイム」から「セッションを再起動する」
# を選択して再度このセルを実行する必要がある。

# Google Colab 上で実行しているかどうかを判定
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

# Google colab で実行している場合必要なライブラリをインストールして word2vec ファイルをダウンロードする
if isColab:
    # Install PyDrive
    !pip install pydrive2
    #!pip install PyDrive2

    #Import modules
    from pydrive2.auth import GoogleAuth
    #from pydrive.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    #Authenticate and create the PyDrive client
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    #Get the Shareable link
    # 2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz: 1Rp3HbDkbpzMg5ehq1ARwCATX8iZAxTgj
    # 2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz: 19BKVOBNHESt1K8725UTM9J3OpqK7YlVb
    downloaded = drive.CreateFile({'id':"1Rp3HbDkbpzMg5ehq1ARwCATX8iZAxTgj"})
    downloaded.GetContentFile('2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz')

    # 2021_05jawiki_hid200_win20_neg20_sgns.bin.gz: 1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s
    # 2021_05jawiki_hid200_win20_neg20_cbow.bin.gz: 1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7
    # 2021_05jawiki_hid128_win10_neg10_sgns.bin.gz: 1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M
    # 2021_05jawiki_hid128_win10_neg10_cbow.bin.gz: 1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb
    # 上記は cbow で訓練済ファイルをダウンロードしています。
    # skip gram モデルで訓練済モデルに変更する場合には，上記情報に従って適宜書き換えてください。

    !pip uninstall numpy thinc spacy --yes
    !pip install --upgrade numpy==1.26.4 --force-reinstall
    !pip install --upgrade gensim==4.3.3
    #!pip install --upgrade numpy==1.26.4 gensim==4.3.3 --force-reinstall

    import numpy as np
    print(f'np.__version__:{np.__version__}')

    import gensim
    print(f'gensim.__version__:{gensim.__version__}')

In [2]:
# 上セルでダウンロードした訓練済 word2vec ファイルを読み込む
word2vec_file='2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
asakawa_w2v = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file,
                                                        encoding='utf-8',
                                                        unicode_errors='replace',
                                                        binary=True)

In [ ]:
# 確認
#help(asakawa_w2v.get_vector)
asakawa_w2v.get_vector('頻度').shape # (200,)  これは word2vec の次元数

In [ ]:
# tlpa のデータを GitHub から取得する
!wget 'https://github.com/ShinAsakawa/ShinAsakawa.github.io/blob/master/tlpa.pkl?raw=true'
!mv 'tlpa.pkl?raw=true' tlpa.pkl

In [8]:
# 取得した tlpa データの読み込み
import pickle
with open('tlpa.pkl','rb') as f:
    tlpa = pickle.load(f)

In [ ]:
 # tlpa の親密度とカテゴリーの表記情報を読み込む
tlpa_fam = [tlpa[k]['Fam'] for k in tlpa.keys()]
tlpa_cat = [tlpa[k]['Cat'] for k in tlpa.keys()]
print(set(tlpa_fam), len(tlpa_fam), set(tlpa_cat), len(tlpa_fam))

In [ ]:
# 印刷して確認
print(tlpa_fam)
print(tlpa_cat)

In [11]:
# 以下の作業は，tlpa データと word2vec のデータが不一致だったので強制的に書き換えています
tlpa[106]['label'] = "刺身"
tlpa[116]['label'] = "すりこ木"
tlpa[123]['label'] = "ぶどう"
tlpa[139]['label'] = "畳"
tlpa[162]['label'] = "タイヤキ"
tlpa[188]['label'] = "タツノオトシゴ"

In [ ]:
# ここがポイントで，tlpa のデータと word2vec のデータを結合します
w2v_vect = np.zeros([200,200])
for k in tlpa.keys():
    word = tlpa[k]['label']
    w2v_vect[k-1] = np.copy(asakawa_w2v[word])
    if k % 10 == 0:
        print('{0:03d} {1:s}'.format(k, word))
    else:
        print('{0:03d} {1:s}'.format(k, word), end=", ")


In [ ]:
# tSNE の前段階として主成分分析を実施します
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns

tlpa_pca = PCA(n_components=3)
tlpa_pca_result = tlpa_pca.fit_transform(w2v_vect)
pca1 = tlpa_pca_result[:,0]
pca2 = tlpa_pca_result[:,1]
pca3 = tlpa_pca_result[:,2]
print('Explained variation per principal component: {}'.format(tlpa_pca.explained_variance_ratio_))

In [17]:
# 結果の再現性を保証するため，乱数の種を指定します。この作業は省略可です
np.random.seed(42)
#rndperm = np.random.permutation(w2v_vect.shape[0])

In [ ]:
# 主成分分析の結果を画面に表示
x_inch = 7; y_inch = 7
fig = plt.figure(figsize=(x_inch, y_inch))
plt.scatter(pca1,pca2)

In [19]:
# 主成分分析結果をカテゴリごとに色分けして描画
df = pd.DataFrame(tlpa_pca_result, columns=["pca1", "pca2", "pca3"])
#help(sns.scatterplot)
df["fam"] = tlpa_fam; df["cat"] = tlpa_cat

T:乗り物, C:色, P:植物, F:加工食品, E:建造物, D:道具, V:野菜果物，B:身体部位, I:屋内部位, A:動物

In [ ]:
plt.figure(figsize=(12,12))
ax = sns.scatterplot(x="pca2", y="pca1", data=df, hue="cat", size="fam", sizes=(100,300)) # , legend="full")
#T:乗り物, C:色, P:植物, F:加工食品, E:建造物, D:道具, V:野菜果物，B:身体部位, I:屋内部位, A:動物

In [21]:
# 上記の図を pdf ファイルとして保存
fig = ax.get_figure()
fig.savefig('tlpa_pca2-3.pdf')

# ここから先が tSNE の計算

In [24]:
#time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=500)  # tSNE の定義
tsne_results = tsne.fit_transform(w2v_vect)  # tSNE による word2vec の可視化
#print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 200 samples in 0.001s...
[t-SNE] Computed neighbors for 200 samples in 0.009s...
[t-SNE] Computed conditional probabilities for sample 200 / 200
[t-SNE] Mean sigma: 12.173744
[t-SNE] KL divergence after 250 iterations with early exaggeration: 67.403603
[t-SNE] KL divergence after 500 iterations: 0.620870


In [ ]:
# tSNE 結果の描画
df_subset = df.copy()
#df_subset['pca-one'] = pca_result[:,0]
#df_subset['pca-two'] = pca_result[:,1]
#df_subset['pca-three'] = pca_result[:,2]

df_subset['tSNE1'] = tsne_results[:,0]
df_subset['tSNE2'] = tsne_results[:,1]

plt.figure(figsize=(12,12))
ax = sns.scatterplot(x="tSNE1", y="tSNE2", hue="cat", size="fam",
    data=df_subset, sizes=(120,300)
)
fig = ax.get_figure()
fig.savefig('tlpa_tSNE.pdf')  # 結果の保存